# Data retrieval script

This notebook details the steps used to retrieve data from Lawline and save it to disk.

In [ ]:
import time

import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [ ]:
link_list = []

In [ ]:
options = Options()
options.add_argument("--headless")
options.add_argument("--disable-gpu")
driver = webdriver.Chrome(options=options)

In [ ]:
for _ in range(5):
    max_i = 10146

    for i in range(1, max_i):
        url = f"https://lawline.se/categories?page={i}"
        driver.get(url)
        time.sleep(3)
        page = driver.page_source
        soup = BeautifulSoup(page, "html.parser")
        links = soup.find_all(
            "a", attrs={"class": "d-flex txt_grey flex-wrap"}, href=True
        )

        # Get links from page source
        for link in links:
            link_end = link.get("href")
            if link_end not in link_list:
                link_list.append(link_end)

        if i % 500 == 0:
            str_link_list = str(link_list)

            with open("link_list2.txt", "w") as f:
                f.write(str_link_list)
            print(i)

In [ ]:
str_link_list = str(link_list)

with open("link_list.txt", "w") as f:
    f.write(str_link_list)

In [ ]:
len(link_list)

In [ ]:
def get_fields(soup_in):
    # Date
    soup_date = soup_in.find(
        "span", attrs={"class": "PublicQuestionItem_published_date__v9qn2"}
    ).text

    # Title
    soup_title = soup_in.find(
        "h1", attrs={"class": "PublicQuestionItem_question_title__I4KwB"}
    ).text

    # Question
    soup_q = soup_in.find(
        "span", attrs={"class": "PublicQuestionItem_question_text__dLAHK"}
    ).text

    # Answer
    soup_answer = soup_in.find(
        "span", attrs={"class": "PublicQuestionDetails_answer_text__nhqcw"}
    ).text

    return soup_date, soup_title, soup_q, soup_answer

In [ ]:
id_list = []
date_list = []
text_list = []
title_list = []
issues_list = []

In [ ]:
options = Options()
options.add_argument("--headless")
options.add_argument("--disable-gpu")
driver = webdriver.Chrome(options=options)

In [ ]:
id = 1

for link in link_list:

    url = f"https://lawline.se{link}"
    driver.get(url)
    time.sleep(2)
    page = driver.page_source
    soup = BeautifulSoup(page, "html.parser")

    try:
        soup_date, soup_title, soup_q, soup_answer = get_fields(soup)
    except:
        issues_list.append(link)
        continue

    if soup_date is None or soup_title is None or soup_q is None or soup_answer is None:
        issues_list.append(link)
        continue
    else:
        id_list.extend([id, id])
        date_list.extend([soup_date, soup_date])
        text_list.extend([soup_q, soup_answer])
        title_list.extend([soup_title, soup_title])
        id += 1

    if id % 500 == 0:
        df = pd.DataFrame(
            {"id": id_list, "date": date_list, "text": text_list, "title": title_list}
        )
        df.to_csv("lawline_data.csv", index=False)
        print(id)

In [ ]:
df = pd.DataFrame(
    {"id": id_list, "date": date_list, "text": text_list, "title": title_list}
)

df_copy = df.copy()

df_copy = df_copy.drop_duplicates()

In [ ]:
df_copy.to_csv("../dataset/lawline_data.csv", index=False)